In [1]:
import numpy as np
import os
DATASETS='DBLP'
#DATASETS='Movie'
#DATASETS='Foursquare'
walks=np.load(os.path.join('EmbeddingData',DATASETS+'walks.npy'))
#print(walks[0])

In [2]:
def pairs(k,line,dic,word_dic):#k=window_size
    for index,w in enumerate(line):
        c_list=line[max(0,index-k):min(len(line)-1,index+k)]
        for c in c_list:
            if c!=w:
                if str([w,c]) in dic:
                    dic[str([w,c])]+=1
                else:
                    dic[str([w,c])]=1
            else:
                pass
        if w in nword:
            word_dic[w]+=1
        else:
            word_dic[w]=1
    return 0

In [3]:
import threading
from tqdm import tqdm

nword={}
pairs_dic={}
for line in tqdm(walks):
    threading.Thread(target=pairs, args=(10,line,pairs_dic,nword)).start()
print(len(nword))

100%|██████████| 15649/15649 [01:50<00:00, 141.99it/s]

15649


In [5]:
np.save(os.path.join('EmbeddingData',DATASETS+'w_c_mat.npy'),pairs_dic)
np.save(os.path.join('EmbeddingData',DATASETS+'nword.npy'),nword)

In [6]:
import numpy as np
import math
data = np.zeros((len(nword),len(nword)))
for i,row in tqdm(enumerate(nword)):
    for j,col in enumerate(nword):
        if str([row,col]) in pairs_dic:
            data[i][j]=math.log(pairs_dic[str([row,col])]*len(pairs_dic)/(5*nword[row]*nword[col]))
#             if data[i][j]<1e-5:
#                 data[i][j]=0
        else:
            data[i][j]=0                 
data=np.mat(data)
np.save(os.path.join('EmbeddingData',DATASETS+'logM.npy'),data)

15649it [05:05, 51.15it/s]


In [8]:
data=np.load(os.path.join('EmbeddingData',DATASETS+'logM.npy'))

In [11]:
name=[]
for i in tqdm(nword):
    name.append(i)
print(name[0:10])
np.save(os.path.join('EmbeddingData',DATASETS+'new_id'),name)

100%|██████████| 15649/15649 [00:00<00:00, 2000203.06it/s]

['a4059', 'p4058', 'p6427', 't13346', 'p7329', 't11196', 'p9834', 'a1475', 'p9796', 'p9057']


In [12]:
import time
from fbpca import diffsnorms, pca
from numpy import diag  
tag=2
if tag==1:
    u,sigma,vT=np.linalg.svd(data,full_matrices=1,compute_uv=1)
    print(data.shape)
else:
    time_start=time.time()
    (u,sigma,vT) = pca(data, 128, True)
    time_end=time.time()
    print('totally cost',time_end-time_start)

totally cost 1.540910243988037


In [13]:
np.save(os.path.join('EmbeddingData',DATASETS+'emb_u.npy'),np.dot(u,np.sqrt(np.diag(sigma))))
np.save(os.path.join('EmbeddingData',DATASETS+'emb_v.npy'),np.dot(vT.T,np.sqrt(np.diag(sigma))))